## Using Pydantic for Data Validation

In [ ]:
!pip install tinydb pydantic

In [ ]:
from pydantic import BaseModel, EmailStr, conint, ValidationError

## 1. Define the data model
class User(BaseModel):
    username: str
    email: EmailStr
    age: conint(ge=0)

from tinydb import TinyDB, Query
# Initialize the database
db = TinyDB('05.TinyDB.json')
users_table = db.table('users')
print('Table `users` created in `05.TinyDB.json`\n')


## 2. Create
# Data to insert
user_data = {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}

# Validate and insert data
try:
    # Validate data against the User model
    user = User(**user_data)
    
    # Insert validated data into TinyDB
    users_table.insert(user.dict())
    print('User `john_doe` is created!\n')
except ValidationError as e:
    print("Validation Error:", e.json())


## 3. Read
# Retrieve and validate data
UserQuery = Query()
result = users_table.search(UserQuery.username == 'john_doe')

try:
    user = User(**result[0])
    print("Retrieved User:", user)
except ValidationError as e:
    print("Data Validation Error:", e.json())


---

## Using Cerberus for Data Validation

In [1]:
!pip install cerberus

### 1. Define the Cerberus Schema

In [2]:
## 1. Define a Validation Schema with Cerberus
from cerberus import Validator

schema = {
    'username': {'type': 'string', 'empty': False, 'maxlength': 50},
    'email': {'type': 'string', 'empty': False, 'regex': r'^\S+@\S+\.\S+$'},
    'age': {'type': 'integer', 'min': 0}
}

validator = Validator(schema)


### 2. Initialize TinyDB

In [3]:
## 2. Initialize DB
from tinydb import TinyDB

db = TinyDB('05.TinyDB.json')
users_table = db.table('users')

In [ ]:
## 3. Create
## Validate Data Before Insertion

In [4]:
user_data = {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}

if validator.validate(user_data):
    users_table.insert(user_data)
else:
    print("Validation Errors:", validator.errors)


In [5]:
user_data = {'username': 'john_doe', 'email': 'john@example.com', 'age': -30}

if validator.validate(user_data):
    users_table.insert(user_data)
else:
    print("Validation Errors:", validator.errors)


Validation Errors: {'age': ['min value is 0']}


In [6]:
## 4. Read and Validate
from tinydb import Query

UserQuery = Query()
result = users_table.search(UserQuery.username == 'john_doe')

for user_data in result:
    if validator.validate(user_data):
        print("Retrieved User:", user_data)
    else:
        print("Data Validation Error:", validator.errors)


Retrieved User: {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}
Retrieved User: {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}
Retrieved User: {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}
Retrieved User: {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}
Retrieved User: {'username': 'john_doe', 'email': 'john@example.com', 'age': 30}


### 3. Implement Validation During Update

When performing an update, you'll need to:
* Retrieve the existing record(s) that match your query.
* Merge the updates with the existing data.
* Validate the merged data.
* Apply the update if validation passes.